In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, Subset
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms

# ========================
# Dataset path
# ========================
data_dir = r"C:\Dataset_ClassWise"

# Load ImageFolder dataset
full_dataset = datasets.ImageFolder(root=data_dir)

# ========================
# Transforms
# ========================
base_transform = transforms.Compose([
    transforms.Resize((224,224))
])

# Separate augmentation for straight-looking faces
def get_augment_transform(label):
    if label == "looking_straight":
        # Minimal augmentation for straight-looking
        return transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
        ])
    else:
        # Aggressive augmentation for other classes
        return transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        ])

to_tensor_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

# ========================
# Balanced Dataset Class
# ========================
class BalancedDataset(Dataset):
    def __init__(self, dataset, target_size=500):
        self.dataset = dataset
        self.target_size = target_size
        self.indices = []
        self._balance_dataset()

    def _balance_dataset(self):
        # Count original samples
        class_counts = {cls: 0 for cls in self.dataset.classes}
        for _, label in self.dataset.samples:
            class_counts[self.dataset.classes[label]] += 1
        print("Original Counts:", class_counts)

        # Collect indices by class
        class_indices = {cls: [] for cls in self.dataset.classes}
        for idx, (_, label) in enumerate(self.dataset.samples):
            class_indices[self.dataset.classes[label]].append(idx)

        balanced_indices = []
        for cls in self.dataset.classes:
            idxs = class_indices[cls]
            if len(idxs) >= self.target_size:
                # Undersample
                balanced_indices.extend(random.sample(idxs, self.target_size))
            else:
                # Oversample
                balanced_indices.extend(idxs)
                while len([i for i in balanced_indices if i in idxs]) < self.target_size:
                    balanced_indices.append(random.choice(idxs))
        self.indices = balanced_indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        img_path, label = self.dataset.samples[self.indices[idx]]
        img = Image.open(img_path).convert("RGB")
        img = base_transform(img)
        # Apply class-specific augmentation if oversampled
        if self.indices.count(self.indices[idx]) > 1:
            img = get_augment_transform(self.dataset.classes[label])(img)
        img = to_tensor_transform(img)
        return img, label

# ========================
# Create & save balanced dataset
# ========================
balanced_dataset = BalancedDataset(full_dataset, target_size=500)
torch.save({
    'indices': balanced_dataset.indices,
    'samples': full_dataset.samples,
    'classes': full_dataset.classes
}, r"C:\Dataset_ClassWise\balanced_dataset_new.pt")  # different file name

print("Balanced dataset created and saved. Total samples:", len(balanced_dataset))


Original Counts: {'looking_down': 49, 'looking_left': 24, 'looking_right': 30, 'looking_straight': 24, 'looking_up': 49, 'multiple_faces': 12}
Balanced dataset created and saved. Total samples: 3000


In [3]:
import torch
from torch.utils.data import Dataset, Subset
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import transforms

# ========================
# Load saved balanced dataset
# ========================
data = torch.load(r"C:\Dataset_ClassWise\balanced_dataset_new.pt")

# ========================
# Define transforms
# ========================
base_transform = transforms.Compose([
    transforms.Resize((224,224))
])

to_tensor_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

# Class-specific augmentation function
def get_augment_transform(label):
    if label == "looking_straight":
        return transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        ])

# ========================
# Dataset Wrapper using saved indices
# ========================
class SavedBalancedDataset(Dataset):
    def __init__(self, saved_data):
        self.indices = saved_data['indices']
        self.samples = saved_data['samples']
        self.classes = saved_data['classes']

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        img_path, label = self.samples[self.indices[idx]]
        img = Image.open(img_path).convert("RGB")
        img = base_transform(img)
        img = get_augment_transform(self.classes[label])(img)  # class-specific augmentation
        img = to_tensor_transform(img)
        return img, label

# ========================
# Load dataset
# ========================
balanced_dataset = SavedBalancedDataset(data)
print("Balanced Dataset Size:", len(balanced_dataset))
print("Classes:", balanced_dataset.classes)

# ========================
# Train/Validation Split
# ========================
labels = [balanced_dataset.samples[idx][1] for idx in balanced_dataset.indices]

indices = list(range(len(balanced_dataset)))
train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

train_dataset = Subset(balanced_dataset, train_idx)
val_dataset = Subset(balanced_dataset, val_idx)

print("Train samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))


Balanced Dataset Size: 3000
Classes: ['looking_down', 'looking_left', 'looking_right', 'looking_straight', 'looking_up', 'multiple_faces']
Train samples: 2400
Validation samples: 600


In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import models
import time

# ========================
# Device
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ========================
# DataLoaders
# ========================
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

# ========================
# Load pretrained MobileNetV2
# ========================
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)

# Replace classifier for 6 classes
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 6)
model = model.to(device)

# ========================
# Loss & optimizer
# ========================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ========================
# Training Loop
# ========================
num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    start_time = time.time()
    
    # Training
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    val_loss /= total
    val_acc = correct / total

    # Save best model (new file)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), r"C:\Dataset_ClassWise\mobilenetv2_best_new.pth")

    end_time = time.time()
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} "
          f"Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f} "
          f"Time: {end_time-start_time:.1f}s")

print(f"Training complete! Best validation accuracy: {best_val_acc:.4f}")


Using device: cpu
Epoch [1/10] Train Loss: 1.1323 Train Acc: 0.6683 Val Loss: 0.6245 Val Acc: 0.8533 Time: 344.3s
Epoch [2/10] Train Loss: 0.3730 Train Acc: 0.9150 Val Loss: 0.1678 Val Acc: 0.9567 Time: 313.3s
Epoch [3/10] Train Loss: 0.1221 Train Acc: 0.9675 Val Loss: 0.0776 Val Acc: 0.9767 Time: 312.8s
Epoch [4/10] Train Loss: 0.0662 Train Acc: 0.9796 Val Loss: 0.0433 Val Acc: 0.9800 Time: 269.3s
Epoch [5/10] Train Loss: 0.0539 Train Acc: 0.9817 Val Loss: 0.0340 Val Acc: 0.9833 Time: 284.8s
Epoch [6/10] Train Loss: 0.0407 Train Acc: 0.9879 Val Loss: 0.0157 Val Acc: 0.9967 Time: 298.6s
Epoch [7/10] Train Loss: 0.0372 Train Acc: 0.9850 Val Loss: 0.0217 Val Acc: 0.9867 Time: 301.7s
Epoch [8/10] Train Loss: 0.0326 Train Acc: 0.9892 Val Loss: 0.0118 Val Acc: 0.9967 Time: 312.8s
Epoch [9/10] Train Loss: 0.0278 Train Acc: 0.9925 Val Loss: 0.0182 Val Acc: 0.9900 Time: 297.9s
Epoch [10/10] Train Loss: 0.0180 Train Acc: 0.9908 Val Loss: 0.0130 Val Acc: 0.9967 Time: 222.7s
Training complete! Be

In [6]:
import torch
from PIL import Image
from torchvision import transforms, models

# ========================
# Device
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ========================
# Classes
# ========================
classes = ['looking_down', 'looking_left', 'looking_right', 'looking_straight', 'looking_up', 'multiple_faces']

# ========================
# Load trained model
# ========================
model = models.mobilenet_v2(weights=None)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, len(classes))
model.load_state_dict(torch.load(r"C:\Dataset_ClassWise\mobilenetv2_best_new.pth", map_location=device))
model = model.to(device)
model.eval()
print("Model loaded successfully!")

# ========================
# Preprocessing function
# ========================
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ========================
# Load image
# ========================
img_path = r"C:\Dataset_ClassWise\looking_right\test.jpg"  # replace with your test image path
img = Image.open(img_path).convert("RGB")
img_tensor = preprocess(img).unsqueeze(0).to(device)  # add batch dimension

# ========================
# Predict
# ========================
with torch.no_grad():
    outputs = model(img_tensor)
    _, predicted = torch.max(outputs, 1)
    print("Predicted Eye Gaze Class:", classes[predicted.item()])


Using device: cpu
Model loaded successfully!
Predicted Eye Gaze Class: looking_right


In [7]:
import cv2
import torch
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import time

# ========================
# Device
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ========================
# Classes
# ========================
classes = ['looking_down', 'looking_left', 'looking_right', 'looking_straight', 'looking_up', 'multiple_faces']

# ========================
# Load model
# ========================
model = models.mobilenet_v2(weights=None)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, len(classes))
model.load_state_dict(torch.load(r"C:\Dataset_ClassWise\mobilenetv2_best_new.pth", map_location=device))
model = model.to(device)
model.eval()
print("Model loaded successfully!")

# ========================
# Preprocess function
# ========================
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# ========================
# Exam rules
# ========================
MAX_CHANCES = 3            # number of allowed violations
MAX_CONTINUOUS = 4.0       # max seconds looking away continuously
WINDOW_TIME = 20.0         # seconds interval to monitor violations

# ========================
# Start Video Capture
# ========================
cap = cv2.VideoCapture(0)  # change index if needed
start_time = time.time()
violation_count = 0
continuous_away_start = None
gaze_log = []

print("Starting video proctoring... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    timestamp = time.time() - start_time

    # Convert frame to PIL and preprocess
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tensor = preprocess(img).unsqueeze(0).to(device)

    # Predict gaze
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        gaze = classes[predicted.item()]

    # Determine if suspicious
    if gaze != "looking_straight":
        if continuous_away_start is None:
            continuous_away_start = timestamp
        continuous_duration = timestamp - continuous_away_start
        reason = "looked away"
    else:
        continuous_duration = 0
        continuous_away_start = None
        reason = "normal"

    # Update violation count if continuous > MAX_CONTINUOUS
    if continuous_duration > MAX_CONTINUOUS:
        violation_count += 1
        continuous_away_start = None  # reset
        reason = f"suspicious > {MAX_CONTINUOUS}s"

    # Append log
    gaze_log.append({
        "Time (s)": round(timestamp, 2),
        "Gaze": gaze,
        "ContinuousAway(s)": round(continuous_duration,2),
        "ViolationCount": violation_count,
        "Reason": reason
    })

    # Display frame with info
    cv2.putText(frame, f"Gaze: {gaze}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, f"Violations: {violation_count}/{MAX_CHANCES}", (10,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    cv2.imshow("Exam Proctoring", frame)

    # Terminate if violations exceed limit
    if violation_count >= MAX_CHANCES:
        print(f"Cheating detected at {timestamp:.2f}s! Reason: {reason}")
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# ========================
# Save log to CSV
# ========================
log_df = pd.DataFrame(gaze_log)
log_df.to_csv(r"C:\Dataset_ClassWise\gaze_log.csv", index=False)
print("Gaze log saved to gaze_log.csv")


Using device: cpu
Model loaded successfully!
Starting video proctoring... Press 'q' to quit.
Cheating detected at 12.61s! Reason: suspicious > 4.0s
Gaze log saved to gaze_log.csv
